# Can we predict machine failures before they happen?

Using machine learning to predict mechanical failure

![image](image.png)

Image from [H.O.Penn](https://www.hopenn.com/blog/what-to-do-when-your-equipment-breaks-down/)

**Creator Notes:**
- Use `light` mode view for optimal reading.
- Plots are interactive, allowing you to explore the data in detail.

# Executive Summary

* LightGBM with RobustScaler achieved high accuracy (98.9%) in predicting machine failures.
* Mechanical stress (torque, cutting force, pressure) are the most important factors for predicting failures.
* A single model might be effective for all machines due to consistent failure patterns across them.
* Investigate systematic issues causing failures (e.g., maintenance practices, environment).
* Continuously monitor and evaluate the deployed LightGBM model's performance.


## I. Background

Manufacturing downtime costs our aerospace and medical components facility both time and money. With three specialized machines producing different sized components, any unexpected failure disrupts production schedules and impacts delivery deadlines. Our current reactive maintenance approach means we fix problems after they occur — leading to longer downtimes and missed deadlines.

## II. Objectives

This project aims to predict machine failures before they happen, enabling our maintenance team to plan repairs proactively. We will develop a predictive model using a year's worth of operational data to identify early warning signs of potential failures. Our key goals are identifying the most reliable failure indicators and determining whether machine-specific models perform better than a general approach.

## III. The data

Our dataset contains daily operational measurements from three production machines spanning one year. Each record includes 13 sensor measurements — from hydraulic pressure to spindle vibration — along with machine identifiers and downtime status. The data comes from critical systems including cooling, hydraulics, and cutting mechanisms. All measurements follow standardized units and are recorded at consistent daily intervals.

| Column Name | Description | Unit | Significance |
|------------|-------------|------|--------------|
| Date | Daily timestamp of readings | YYYY-MM-DD | Tracks temporal patterns and maintenance history |
| Machine_ID | Unique machine identifier | Text | Enables machine-specific analysis and comparisons |
| Assembly_Line_No | Production line location | Integer | Maps physical layout and workflow dependencies |
| Hydraulic_Pressure | Hydraulic system pressure | bar | Indicates fluid power system health |
| Coolant_Pressure | Cooling system pressure | bar | Monitors heat dissipation efficiency |
| Air_System_Pressure | Pneumatic system pressure | bar | Reflects compressed air system status |
| Coolant_Temperature | Cooling system temperature | Celsius | Tracks thermal management effectiveness |
| Hydraulic_Oil_Temperature | Hydraulic fluid temperature | Celsius | Indicates system stress and oil condition |
| Spindle_Bearing_Temperature | Bearing temperature | Celsius | Monitors critical component health |
| Spindle_Vibration | Spindle oscillation | micrometers | Detects mechanical imbalances |
| Tool_Vibration | Cutting tool movement | micrometers | Indicates tool wear and stability |
| Spindle_Speed | Rotational velocity | RPM | Measures cutting performance |
| Voltage | Electrical input | volts | Monitors power supply stability |
| Torque | Rotational force | Nm | Indicates mechanical load |
| Cutting | Tool force | KN | Measures material removal effort |
| Downtime | Operational status | Boolean | Records machine availability |

The company has stored the machine operating data in a single table, available in `'data/machine_downtime.csv'`.

In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Scikit-learn imports
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer

# Metrics and evaluation
from sklearn.metrics import (
   accuracy_score, precision_score, recall_score, f1_score,
   roc_curve, precision_recall_curve, auc, roc_auc_score,
   confusion_matrix, classification_report
)

from sklearn.model_selection import cross_val_score

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Configuration settings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the dataset and display the first 10 rows
data = pd.read_csv('data/machine_downtime.csv')
display(data.head(10))

,Date,Machine_ID,Assembly_Line_No,Hydraulic_Pressure(bar),Coolant_Pressure(bar),Air_System_Pressure(bar),Coolant_Temperature,Hydraulic_Oil_Temperature,Spindle_Bearing_Temperature,Spindle_Vibration,Tool_Vibration,Spindle_Speed(RPM),Voltage(volts),Torque(Nm),Cutting(kN),Downtime
0,31-12-2021,Makino-L1-Unit1-2013,Shopfloor-L1,71.040000,6.933725,6.284965,25.6,46.0,33.4,1.291,26.492,25892.0,335.0,24.055326,3.58,Machine_Failure
1,31-12-2021,Makino-L1-Unit1-2013,Shopfloor-L1,125.330000,4.936892,6.196733,35.3,47.4,34.6,1.382,25.274,19856.0,368.0,14.202890,2.68,Machine_Failure
2,31-12-2021,Makino-L3-Unit1-2015,Shopfloor-L3,71.120000,6.839413,6.655448,13.1,40.7,33.0,1.319,30.608,19851.0,325.0,24.049267,3.55,Machine_Failure
3,31-05-2022,Makino-L2-Unit1-2015,Shopfloor-L2,139.340000,4.574382,6.560394,24.4,44.2,40.6,0.618,30.791,18461.0,360.0,25.860029,3.55,Machine_Failure
4,31-03-2022,Makino-L1-Unit1-2013,Shopfloor-L1,60.510000,6.893182,6.141238,4.1,47.3,31.4,0.983,25.516,26526.0,354.0,25.515874,3.55,Machine_Failure
5,31-03-2022,Makino-L2-Unit1-2015,Shopfloor-L2,137.370000,5.918357,7.228066,5.4,48.0,32.7,0.903,25.597,27613.0,319.0,25.521330,3.55,Machine_Failure
6,31-03-2022,Makino-L1-Unit1-2013,Shopfloor-L1,135.930000,6.560332,6.710999,19.3,48.8,37.4,1.240,32.138,26605.0,438.0,25.454652,3.58,Machine_Failure
7,31-03-2022,Makino-L3-Unit1-2015,Shopfloor-L3,127.715164,5.060709,6.002229,20.8,45.8,37.5,1.125,19.823,14266.0,334.0,34.973004,2.02,No_Machine_Failure
8,31-03-2022,Makino-L3-Unit1-2015,Shopfloor-L3,123.618456,5.074380,6.039524,4.5,51.5,32.1,0.690,16.972,20413.0,278.0,32.519299,2.88,No_Machine_Failure
9,31-03-2022,Makino-L3-Unit1-2015,Shopfloor-L3,134.020000,5.567857,6.733096,14.0,47.9,35.2,0.748,36.601,20504.0,379.0,25.618567,3.93,Machine_Failure


In [3]:
# Display information about the dataset including the data types and non-null counts for each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         2500 non-null   object 
 1   Machine_ID                   2500 non-null   object 
 2   Assembly_Line_No             2500 non-null   object 
 3   Hydraulic_Pressure(bar)      2490 non-null   float64
 4   Coolant_Pressure(bar)        2481 non-null   float64
 5   Air_System_Pressure(bar)     2483 non-null   float64
 6   Coolant_Temperature          2488 non-null   float64
 7   Hydraulic_Oil_Temperature    2484 non-null   float64
 8   Spindle_Bearing_Temperature  2493 non-null   float64
 9   Spindle_Vibration            2489 non-null   float64
 10  Tool_Vibration               2489 non-null   float64
 11  Spindle_Speed(RPM)           2494 non-null   float64
 12  Voltage(volts)               2494 non-null   float64
 13  Torque(Nm)        

## IV. EDA Summary

For a comprehensive understanding of the data analysis, readers are encouraged to review my [previous report](https://www.datacamp.com/datalab/w/7df48228-d6f8-4c57-a25b-061591934c38), which provides detailed exploratory data analysis. Here are the key findings from [How do machines behave before downtime?](https://www.datacamp.com/datalab/w/7df48228-d6f8-4c57-a25b-061591934c38) that inform our current modeling approach:

Correlation Analysis:
- Sensor measurements show independence (correlation coefficients < 0.25)
- All variables retained for modeling due to their independent predictive potential
- Temporal features include day_of_week and is_weekday indicators

Temporal Patterns:
- Peak failure incidents observed in March-April 2022
- Weekday failures occur 3x more frequently than weekend failures
- Strong correlation between production schedules and machine reliability

High-Predictive Variables:
- Hydraulic pressure (bimodal distribution)
- Tool vibration measurements
- Spindle speed readings
- Cutting force metrics
- Torque values
- Coolant pressure levels

Preprocessing Strategy:
1. Missing Value Treatment:
   - Mean imputation for normal distributions
   - KNN imputation for bimodal distributions

2. Outlier Handling:
   - Retain outliers as they represent valid operational states
   - Evaluate multiple scaling methods
   - RobustScaler anticipated as optimal choice due to outlier presence

3. Class Balance:
   - No class imbalance treatment needed for 'downtime' variable

4. Feature Engineering:
   - Initial approach without feature engineering due to low multicollinearity
   - Will reassess based on baseline model performance

Modeling Approach:
1. Baseline: Logistic Regression
2. Advanced Models:
   - Gradient Boosting (primary candidate for non-linear relationships)
   - Random Forest (alternative for handling outliers and interactions)

This structured approach will guide our model development and evaluation process.

## V. Data Preparation

Before preprocessing, the dataset was prepared by standardizing column names to snake_case for consistency, expanding date features to extract relevant temporal information (e.g., day of week) for potential model improvement, and mapping the target variable.

In [4]:
# Replace parentheses with underscores and convert to lowercase
data.columns = [col.replace('(', '_').replace(')', '').lower() for col in data.columns]

# Convert to datetime to use .dt accessor
data['date'] = pd.to_datetime(data['date'] )

# Extract day of the week
data['day_of_week'] = data['date'].dt.strftime('%A')

# Create boolean column to check if is weekday
data['is_weekday'] = data['date'].dt.weekday < 5

# Define ordered categories for days
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Convert day to ordered categorical variable
data['day_of_week'] = pd.Categorical(data['day_of_week'], categories=day_order, ordered=True)

# Convert the target variable to binary
data['downtime'] = data['downtime'].map({
    'Machine_Failure': 1,
    'No_Machine_Failure': 0})

In [5]:
# Check encoding
print("Target value counts:")
print(data['downtime'].value_counts())
print("\nTarget unique values:")
print(data['downtime'].unique())

Target value counts:
downtime
1    1265
0    1235
Name: count, dtype: int64

Target unique values:
[1 0]


As observed, our target class distribution is balanced.

### 2. Train-Test Split

With only 2,500 records, we opt for a simple train-test split without validation. We allocate 80% (2,000 samples) for training and 20% (500 samples) for testing. A validation set is unnecessary for datasets under 10,000 records as it would reduce our training data significantly.

In [6]:
TARGET = 'downtime'
TEST_SIZE = 0.2
RANDOM_STATE = 1

# Separate features (X) and target (y)
X = data.drop(['date', TARGET], axis=1) # Drop date and target
y = data[TARGET] 

# Print dataset overview
print("Dataset Overview:")
print(f"Total samples: {X.shape[0]}")
print(f"Features: {X.shape[1]}")
print("\nTarget distribution:")
print(y.value_counts(normalize=True))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

print("\nClass Distribution:")
print(f"  Training set:\n{pd.Series(y_train).value_counts(normalize=True).to_string()}") #to_string for better formatting
print(f"  Test set:\n{pd.Series(y_test).value_counts(normalize=True).to_string()}") #to_string for better formatting


Dataset Overview:
Total samples: 2500
Features: 16

Target distribution:
downtime
1    0.506
0    0.494
Name: proportion, dtype: float64

Class Distribution:
  Training set:
downtime
1    0.506
0    0.494
  Test set:
downtime
1    0.506
0    0.494


The test set is used exclusively for final model evaluation to avoid data leakage. We set stratify=y to maintain the original class distribution (51% failures, 49% non-failures) in both splits, ensuring balanced representation of machine failures.
The random_state=1 ensures reproducibility of our results. Time-related features (dates) are dropped as they don't contribute to the prediction task.

**Both training and test sets maintain identical class distributions, confirming our stratification was successful.**

### 3. Data Preprocessing Strategy

Our data preprocessing addresses three key aspects: missing values, feature scaling, and categorical encoding.

Missing Value Treatment:
1. Mean imputation for normally distributed sensors (coolant temperature, hydraulic oil temperature, air pressure, bearing temperature, spindle vibration, voltage) because values naturally cluster around the mean.
2. KNN imputation (k=5) for bimodal features (hydraulic pressure, tool vibration, spindle speed, cutting force, torque, coolant pressure) to preserve their distinct working/failure states.

Feature Scaling Comparison:
- StandardScaler: Centers and scales data to unit variance
- MinMaxScaler: Scales features to fixed range [0,1]
- RobustScaler: Scales using statistics robust to outliers

Categorical Encoding:
- OneHotEncoder for categorical variables (machine_id, assembly_line_no, is_weekday, day_of_week) since order doesn't matter
- Sparse=False for dense matrix output

In [7]:
def fit_preprocessor(X_train: pd.DataFrame, scaling_method: str = 'standard'):
    """
    Fit preprocessing steps on training data only.
    Returns fitted transformers.
    """
    # Define feature groups
    normal_features = [
        'coolant_temperature', 'hydraulic_oil_temperature',
        'air_system_pressure_bar', 'spindle_bearing_temperature',
        'spindle_vibration', 'voltage_volts'
    ]
    
    bimodal_features = [
        'hydraulic_pressure_bar', 'tool_vibration', 
        'spindle_speed_rpm', 'cutting_kn',
        'torque_nm', 'coolant_pressure_bar'
    ]
    
    categorical_cols = ['machine_id', 'assembly_line_no', 'is_weekday', 'day_of_week']
    
    # Initialize transformers
    mean_imputer = SimpleImputer(strategy='mean')
    knn_imputer = KNNImputer(n_neighbors=5)
    
    if scaling_method == 'standard':
        scaler = StandardScaler()
    elif scaling_method == 'minmax':
        scaler = MinMaxScaler()
    elif scaling_method == 'robust':
        scaler = RobustScaler()
    else:
        raise ValueError("Scaling method must be 'standard', 'minmax', or 'robust'")
        
    encoder = OneHotEncoder(sparse_output=False)
    
    # Fit transformers on training data
    mean_imputer.fit(X_train[normal_features])
    knn_imputer.fit(X_train[bimodal_features])
    scaler.fit(X_train[normal_features + bimodal_features])
    encoder.fit(X_train[categorical_cols])
    
    return {
        'mean_imputer': mean_imputer,
        'knn_imputer': knn_imputer,
        'scaler': scaler,
        'encoder': encoder,
        'normal_features': normal_features,
        'bimodal_features': bimodal_features,
        'categorical_cols': categorical_cols
    }

def transform_data(X: pd.DataFrame, preprocessor: dict) -> pd.DataFrame:
    """
    Transform data using fitted preprocessors.
    """
    X_processed = X.copy()
    
    # Apply transformations
    X_processed[preprocessor['normal_features']] = preprocessor['mean_imputer'].transform(
        X[preprocessor['normal_features']]
    )
    X_processed[preprocessor['bimodal_features']] = preprocessor['knn_imputer'].transform(
        X[preprocessor['bimodal_features']]
    )
    
    numeric_cols = preprocessor['normal_features'] + preprocessor['bimodal_features']
    X_processed[numeric_cols] = preprocessor['scaler'].transform(X_processed[numeric_cols])
    
    cat_cols_names = preprocessor['encoder'].get_feature_names_out(preprocessor['categorical_cols'])
    cat_encoded = pd.DataFrame(
        preprocessor['encoder'].transform(X[preprocessor['categorical_cols']]),
        columns=cat_cols_names,
        index=X.index
    )
    
    return pd.concat([X_processed.drop(preprocessor['categorical_cols'], axis=1), cat_encoded], axis=1)

def preprocess_data(X_train: pd.DataFrame, 
                  X_test: pd.DataFrame,
                  scaling_method: str = 'standard') -> tuple:
   """
   Preprocess datasets using transformers fitted only on training data.
   """
   # Fit preprocessor on training data only 
   preprocessor = fit_preprocessor(X_train, scaling_method)
   
   # Transform datasets
   X_train_processed = transform_data(X_train, preprocessor)
   X_test_processed = transform_data(X_test, preprocessor)
   
   return X_train_processed, X_test_processed

**All preprocessing steps are fitted only on training data to prevent data leakage. The same transformations are then applied to test data.**

## VI. Baseline Model

This function, `train_baseline_model`, serves as a baseline evaluation for logistic regression performance. It trains a logistic regression model with different feature scaling methods (`standard`, `minmax`, or `robust`) to assess the impact of scaling on model performance. The function preprocesses the data using the specified scaling method, trains the model, and then calculates various performance metrics like accuracy, precision, recall, F1-score, and AUC-ROC. Finally, it prints the results and generates visualizations including a confusion matrix and ROC curve to provide a comprehensive evaluation of the model's performance with the chosen scaling method. The function also returns the trained model and the calculated metrics for further analysis.


In [8]:
def train_baseline_model(X_train, X_test, y_train, y_test, scaling_method='standard'):
   """
   Train baseline logistic regression with different scaling methods.
   Returns performance metrics and plots.
   """
   # Preprocess with specified scaler
   X_train_processed, X_test_processed = preprocess_data(
       X_train, X_test, scaling_method=scaling_method
   )

   # Train logistic regression
   model = LogisticRegression(random_state=42, max_iter=1000)
   model.fit(X_train_processed, y_train)

   # Get predictions 
   y_pred = model.predict(X_test_processed)
   y_pred_proba = model.predict_proba(X_test_processed)[:,1]
   
   # Calculate metrics
   metrics = {
       'Accuracy': accuracy_score(y_test, y_pred),
       'Precision': precision_score(y_test, y_pred),
       'Recall': recall_score(y_test, y_pred),
       'F1': f1_score(y_test, y_pred),
       'AUC': roc_auc_score(y_test, y_pred_proba)
   }

   # Print results
   print(f"\nResults with {scaling_method} scaling:")
   for metric, value in metrics.items():
       print(f"{metric}: {value:.3f}")

   # Plot confusion matrix and ROC curve
   fig = make_subplots(rows=1, cols=2, subplot_titles=['Confusion Matrix', 'ROC Curve'])
   
   # Confusion matrix
   cm = confusion_matrix(y_test, y_pred)
   fig.add_trace(
       go.Heatmap(z=cm, text=cm, texttemplate="%{text}", colorscale='Blues'),
       row=1, col=1
   )

   # ROC curve 
   fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
   fig.add_trace(
       go.Scatter(x=fpr, y=tpr, name=f'AUC={metrics["AUC"]:.3f}'),
       row=1, col=2
   )

   fig.update_layout(
       title=f'Model Performance with {scaling_method} Scaling',
       height=400
   )
   fig.show()

   return model, metrics

In [9]:
# Test different scalers
scaling_methods = ['standard', 'minmax', 'robust']
results = {}

for scaling in scaling_methods:
   model, metrics = train_baseline_model(X_train, X_test, y_train, y_test, scaling)
   results[scaling] = metrics

# Compare scaling methods
results_df = pd.DataFrame(results).round(3)
print("\nScaling Methods Comparison:")
print(results_df)


Results with standard scaling:
Accuracy: 0.862
Precision: 0.854
Recall: 0.877
F1: 0.865
AUC: 0.927



Results with minmax scaling:
Accuracy: 0.858
Precision: 0.853
Recall: 0.870
F1: 0.861
AUC: 0.925



Results with robust scaling:
Accuracy: 0.864
Precision: 0.854
Recall: 0.881
F1: 0.868
AUC: 0.927



Scaling Methods Comparison:
           standard  minmax  robust
Accuracy      0.862   0.858   0.864
Precision     0.854   0.853   0.854
Recall        0.877   0.870   0.881
F1            0.865   0.861   0.868
AUC           0.927   0.925   0.927


The logistic regression baseline model using RobustScaler achieved 86.4% accuracy, indicating strong fundamental predictive capability. All scaling methods performed similarly, suggesting our features maintain their predictive power across different scaling approaches.

To improve upon this baseline, we will evaluate more sophisticated models. Random Forest and Gradient Boosting algorithms (XGBoost, LightGBM) can capture complex non-linear relationships between sensor measurements. Support Vector Machines (SVM) will help identify complex decision boundaries. K-Nearest Neighbors (KNN) will leverage pattern-based classification. Decision Trees will provide an interpretable comparison point.

**Our high baseline scores (F1: 86.8%, AUC-ROC: 0.927) suggest these advanced models could further enhance prediction accuracy by identifying subtle patterns in sensor behavior before failures occur.**

## V. Model Selection

After establishing our baseline with logistic regression, we evaluate several machine learning models to identify the most effective classifier for machine failure prediction. We compare seven different algorithms: **Logistic Regression, Random Forest, Support Vector Machine (SVM), K-Nearest Neighbors (KNN), Decision Tree, XGBoost, and LightGBM. Each model is tested with three scaling methods (standard, minmax, and robust) to ensure optimal preprocessing.**

The comparison tracks two key metrics:
- Model accuracy to assess prediction performance
- Training time to evaluate computational efficiency

This systematic evaluation will help identify the best model-scaler combination for our predictive maintenance system.


In [16]:
import time

# Initialize models to compare
models = {
   'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE),
   'Random Forest': RandomForestClassifier(random_state=RANDOM_STATE),
   'SVM': SVC(random_state=RANDOM_STATE, probability=True),
   'KNN': KNeighborsClassifier(),
   'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
   'XGBoost': XGBClassifier(random_state=RANDOM_STATE),
   'LightGBM': LGBMClassifier(random_state=RANDOM_STATE, verbose=-1)
}

# Initialize scalers 
scaling_methods = ['standard', 'minmax', 'robust']

# Store results
results = []

# Compare models with different scalers
for scaling in scaling_methods:
   print(f"\nTesting {scaling} scaling:")
   
   # Preprocess data
   X_train_scaled, X_test_scaled = preprocess_data(
       X_train, X_test, scaling_method=scaling
   )
   
   # Train and evaluate each model
   for name, model in models.items():
       start_time = time.time()
       model.fit(X_train_scaled, y_train)
       train_time = time.time() - start_time
       
       # Get predictions
       y_pred = model.predict(X_test_scaled)
       
       results.append({
           'Scaler': scaling,
           'Model': name,
           'Accuracy': accuracy_score(y_test, y_pred),
           'Training Time': train_time
       })
    
# Create results dataframe sorted by accuracy
results_df = pd.DataFrame(results).sort_values('Accuracy', ascending=False)
print("\nModel Performance Summary:")
results_df

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 38)

**Top Performance:**

- LightGBM dominates with 99.8% accuracy (RobustScaler)
- XGBoost follows at 99.4% across all scalers
- Random Forest achieves 98.6% consistently

**Scaling Impact:**

- LightGBM: Best with RobustScaler (99.8%)
- XGBoost: Consistent across scalers (99.4%)
- Random Forest: Stable performance (98.6%)

**Training Efficiency:**

- Fastest: Decision Tree (~0.017s)
- Moderate: XGBoost (~0.082s)
- Slowest: Random Forest (~0.49s)

**Evaluate LightGBM with cross validation to check if it is not overfitting.**

## VII. Best Model Validation

In [25]:
best_scaler = 'robust'
best_model_name = 'LightGBM'

# Test LightGBM with cross-validation
best_model = models[best_model_name]
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=10)

print("Cross-validation scores:", cv_scores)
print(f"Average CV Score: {cv_scores.mean():.3f} (+/- {cv_scores.std() * 2:.3f})")

Cross-validation scores: [0.985 0.985 0.995 0.99  0.99  0.99  0.99  0.99  0.995 0.985]
Average CV Score: 0.989 (+/- 0.007)


LightGBM with RobustScaler demonstrates excellent and consistent performance, achieving a high average cross-validation accuracy of 98.9% (±0.7%). This strong performance across all folds suggests robustness and a low risk of overfitting. Further analysis of learning curves and feature importances will provide deeper insights.

In [17]:
# Preprocess data with best scaler
X_train_best, X_test_best = preprocess_data(
    X_train, 
    X_test, 
    scaling_method=best_scaler.lower()
)

# Get the best model
best_model = models[best_model_name]
best_model.fit(X_train_best, y_train)

# Get predictions
y_pred = best_model.predict(X_test_best)
y_pred_proba = best_model.predict_proba(X_test_best)[:,1]


In [20]:
def plot_classification_metrics(y_test, y_pred, y_pred_proba, best_model_name, best_scaler):
  """
  Create classification metrics visualization with proper spacing between subplots.
  """
  # Create subplots
  fig = make_subplots(
      rows=1, cols=3,
      subplot_titles=(
          'Confusion Matrix',
          'ROC Curve', 
          'Precision-Recall Curve'
      ),
      horizontal_spacing=0.15  # Add spacing between columns
  )

  # Confusion Matrix
  cm = confusion_matrix(y_test, y_pred)
  fig.add_trace(
      go.Heatmap(
          z=cm,
          text=cm,
          texttemplate="%{text}",
          x=['No', 'Yes'],
          y=['No', 'Yes'],
          colorscale='Blues',
          showscale=False
      ),
      row=1, col=1
  )

  # ROC Curve
  fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
  auc_score = auc(fpr, tpr)
  
  fig.add_trace(
      go.Scatter(
          x=fpr, 
          y=tpr,
          name=f'AUC={auc_score:.3f}'
      ),
      row=1, col=2
  )
  fig.add_trace(
      go.Scatter(
          x=[0, 1],
          y=[0, 1],
          line=dict(dash='dash'),
          name='Random'
      ),
      row=1, col=2
  )

  # Precision-Recall Curve
  precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
  pr_auc = auc(recall, precision)
  
  fig.add_trace(
      go.Scatter(
          x=recall,
          y=precision,
          name=f'PR AUC={pr_auc:.3f}'
      ),
      row=1, col=3
  )

  # Update layout with margins and spacing
  fig.update_layout(
      height=400,
      title=f'Model Performance: {best_model_name} with {best_scaler} scaling',
      showlegend=False,
      margin=dict(l=50, r=50, t=80, b=50),  # Add margins
      paper_bgcolor='white',
      plot_bgcolor='white'
  )

  # Update axes labels with padding
  fig.update_xaxes(
      title_text="Predicted", 
      row=1, col=1,
      title_standoff=20  # Add padding between axis and label
  )
  fig.update_yaxes(
      title_text="Actual", 
      row=1, col=1,
      title_standoff=20
  )
  
  fig.update_xaxes(
      title_text="False Positive Rate", 
      row=1, col=2,
      title_standoff=20
  )
  fig.update_yaxes(
      title_text="True Positive Rate", 
      row=1, col=2,
      title_standoff=20
  )
  
  fig.update_xaxes(
      title_text="Recall", 
      row=1, col=3,
      title_standoff=20
  )
  fig.update_yaxes(
      title_text="Precision", 
      row=1, col=3,
      title_standoff=20
  )

  fig.show()

  # Print metrics
  print("\nClassification Report:")
  print(classification_report(y_test, y_pred))

In [21]:
# Plot metrics for best model
plot_classification_metrics(
    y_test=y_test,
    y_pred=y_pred,
    y_pred_proba=y_pred_proba,
    best_model_name=best_model_name,
    best_scaler=best_scaler
)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       247
           1       1.00      1.00      1.00       253

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



The model's performance is further validated by the ROC and PR curves, which are consistent with its high accuracy. The confusion matrix confirms this, showing only a single error. 

In [26]:
def plot_feature_importance(best_model, X_train, best_model_name):
    """
    Plot normalized feature importance for the best model.
    """
    # Get feature importance
    if hasattr(best_model, 'feature_importances_'):
        importances = best_model.feature_importances_
    elif hasattr(best_model, 'coef_'):
        importances = np.abs(best_model.coef_[0])
    else:
        print("Model doesn't provide feature importance")
        return

    # Normalize importances to sum to 1
    importances = importances / importances.sum()

    # Create and sort dataframe
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': importances
    }).sort_values('importance', ascending=True)

    # Create plot
    fig = go.Figure()
    
    # Plot top 15 features
    fig.add_trace(
        go.Bar(
            y=feature_importance['feature'],
            x=feature_importance['importance'] * 100,  # Convert to percentage
            orientation='h'
        )
    )

    # Update layout
    fig.update_layout(
        title=f'Feature Importance: {best_model_name}',
        xaxis_title='Importance (%)',
        yaxis_title='Features',
        height=600,
        width=900,
        margin=dict(l=50, r=50, t=80, b=50),
        showlegend=False
    )

    fig.show()

# Call the function
plot_feature_importance(best_model, X_train_best, best_model_name)

Feature importance reveals that the model utilizes all sensor data, but relies most heavily on torque (20%). This reinforces the importance of monitoring mechanical stress (torque, cutting force) and pressure to predict machine failure, as these are far more informative than time or location.

## VIII. Exporting the Best Model

To ensure reproducibility and facilitate deployment, the trained best model is exported as a serialized object. This allows the model to be easily loaded and used in subsequent analyses or applications without requiring retraining.

In [27]:
import joblib

if best_model:
    filename_model = 'best_model.joblib'
    joblib.dump(best_model, filename_model)
    print(f"Best Model saved to {filename_model}")
else:
    print("No best model to export")

if results_df is not None and not results_df.empty:  # Check if DataFrame is valid
    filename_results = 'model_results.csv'
    results_df.to_csv(filename_results, index=False)  # index=False prevents writing row indices
    print(f"Model results saved to {filename_results}")
else:
    print("No results DataFrame to export")

Best Model saved to best_model.joblib
Model results saved to model_results.csv


In [28]:
def load_model(filename):
    """Loads a model from a joblib file.

    Args:
        filename (str): The path to the joblib file.

    Returns:
        object: The loaded model, or None if an error occurs.
        str: An error message if an error occurs, or None if no error.
    """
    try:
        loaded_model = joblib.load(filename)
        print(f"Model loaded from {filename}")
        return loaded_model, None  # Return the model and no error
    except FileNotFoundError:
        error_message = f"Error: File not found at {filename}"
        print(error_message)
        return None, error_message
    except Exception as e:  # Catch other potential errors
        error_message = f"Error loading model: {e}"
        print(error_message)
        return None, error_message


# Example usage:
model_filename = 'best_model.joblib'  # Replace with your actual filename
loaded_model, error = load_model(model_filename)

if loaded_model:
    # Use the loaded model for predictions or further analysis
    # Example:
    # predictions = loaded_model.predict(new_data)
    print("Model loaded successfully. Ready for use.")
else:
    print(f"Failed to load model. Error: {error}")

Model loaded from best_model.joblib
Model loaded successfully. Ready for use.


## IX. Conclusion 

This analysis successfully trained and evaluated predictive models for machine failure. LightGBM with RobustScaler achieved a remarkable average cross-validation accuracy of 98.9% (±0.7%), demonstrating its potential for real-world application. Feature importance analysis revealed that mechanical stress measurements (torque, cutting force) and pressure play a more critical role in predicting failure compared to temporal or location factors.

## X. Recommendations

Based on the findings, here are key recommendations for optimizing machine maintenance and failure prevention:

1. **Implement a Predictive Maintenance Strategy:** Leverage the LightGBM model for real-time or scheduled predictions of machine failures. This will allow for proactive maintenance interventions before breakdowns occur, minimizing downtime and associated costs.

2. **Prioritize Monitoring of Mechanical Stress:** Focus on closely monitoring sensor data related to mechanical stress, particularly torque, cutting force, and pressure. These factors were identified as the most influential predictors of failure by the model. Implementing real-time monitoring dashboards or alerts based on these specific readings can provide early warnings of potential machine issues.

3. **Consider a Single Predictive Model for All Machines:** The analysis from a previous report [How do machines behave before downtime?](https://www.datacamp.com/datalab/w/7df48228-d6f8-4c57-a25b-061591934c38) revealed consistent failure patterns across all machines, suggesting common factors influencing failures. Given the high accuracy achieved by the single model in this analysis,  investing resources in creating individual machine models might not be necessary.

4. **Further Investigation of Systematic Issues:**  The high performance of a single model for all machines, combined with the findings from the previous report (nearly identical failure rates, similar operating conditions), indicates a need to investigate systematic factors contributing to failures. This could involve analyzing maintenance practices, environmental conditions, or material quality across all machines.

5. **Model Monitoring and Performance Evaluation:** Continuously monitor the real-world performance of the deployed LightGBM model. Over time, machine operating conditions or failure patterns might evolve. Regularly evaluate the model's accuracy and consider retraining if its performance deteriorates.


## Final Note

**_So can we predict machine failures before they happen?_**

Based on the analysis, the answer is **Yes, with high probability**. 

Here's why:

* The LightGBM model achieved a remarkable average cross-validation accuracy of 98.9% (±0.7%). This indicates a very high success rate in predicting machine failures.
* The model focuses heavily on mechanical stress measurements like torque, cutting force, and pressure, which are known indicators of impending failure.
* The analysis suggests common factors influencing failures across machines. This implies that the model can effectively learn from data from all machines and generalize well to predict failures in any of them.

However, it's important to consider a few nuances:

* Machine learning models are not perfect. There's always a chance of a false positive (predicting a failure that doesn't happen) or a false negative (missing an actual failure).
* The model's performance might degrade over time as operating conditions or failure patterns change.

---

If you learned from my modeling and visualization techniques here, do check my entries for [level 1](https://www.datacamp.com/datalab/w/410c2d5d-e90b-4b3e-9979-9a7862fbbef8) and [level 2](https://www.datacamp.com/datalab/w/7df48228-d6f8-4c57-a25b-061591934c38). Connect also with me on [LinkedIn](https://www.linkedin.com/in/jpcurada/) and let's learn by doing!
